# Clovek ne jezi se (aka Mensch aergere dich nicht)



## Agents

Agents are a container for different game policies.


In [1]:
import numpy as np
from random import randint

from clovek_ne_jezi_se.game import Board, Game
from clovek_ne_jezi_se.agent import Player, FurthestAlongAgent

## Game

### Game state

The game state is represented by a triple of arrays: $(waiting, spaces, homes)$.

In [2]:
symbols = ['red', 'yellow', 'green', 'blue']
players = []
for symbol in symbols:
    player = Player(symbol=symbol, number_of_players=4)
    player.initialize_home()
    players.append(player)
game = Game(players)
game.initialize()

# Empty waiting area of one symbol
# TODO: prohibit (or make private) such non-conserving moves, see issue #6
game.set_waiting_count_array('yellow', count=0)

# Occupy one start position
occupied_position = game.get_player('green').get_start_position()
game.set_space_array('blue', occupied_position)

game.set_homes_array('green', 0)
game.set_homes_array('green', 1)

The waiting area is represented by the counts per player.

In [3]:
game.board.waiting_count

{'red': 4, 'yellow': 0, 'green': 4, 'blue': 4}

The spaces (i.e. main board) are represented by a 1-d array.

In [4]:
np.array([[1,2], [3,4] , [5,6]])

array([[1, 2],
       [3, 4],
       [5, 6]])

In [5]:
game.get_spaces_array()

array([-1, -1, -1, -1, -1, -1, -1, -1,  3, -1, -1, -1, -1, -1, -1, -1])

And the home area is represented by a (number of players) x (number of tokens / player) array.

In [6]:
game.get_homes_array()

array([[-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [ 2,  2, -1, -1],
       [-1, -1, -1, -1]])

The `Board` class has more human-readable representation:

In [7]:
game.board


    -------------
    | - | - | blue |
----------------------
| - | - |    | - | - |
--------      -------|
| - |            | - |
--------      -------|
| - | - |    | - | - |
----------------------
    | - | - | - |
    -------------
player red pieces in waiting area: 4 
player red home: - | - | - | -

player yellow pieces in waiting area: 0 
player yellow home: - | - | - | -

player green pieces in waiting area: 4 
player green home: green | green | - | -

player blue pieces in waiting area: 4 
player blue home: - | - | - | -

### Game action

Given a dice roll value $d$, an action is a choice among valid moves of

* *enter_board*: move player piece $p$ from waiting area (or "out" area) onto its `start_position` if $d=6$.
* *advance*:  advance player piece $p \in {0, 1, 2, 3}$ at space $x$ on the board ahead by $d$ to position $d+x$. An advance can be a "home" advance, meaning the move brings a piece into it's home area, or a "space" advance, meaning it stays on the regular board.

An advance is *valid* if no other player piece occupies the space $d+x$, and, in the case of a home advance, the advance does not put the piece beyond the 4 home spots.